In [17]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')
%pip install numpy==1.26.4  pyyaml

In [18]:
df_trips = pd.read_csv('trips_1.5M.csv')
df_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
0,1,2015-06-03 10:41:50,2015-06-03 11:08:33,1,2.80,-73.952888,40.776814,1,N,-73.979034,40.756611,1,17.5,0.0,0.5,4.57,0.0,22.87
1,2,2015-02-14 19:38:55,2015-02-14 20:00:59,1,2.79,-73.995827,40.725353,1,N,-73.984314,40.757389,2,15.0,0.0,0.5,0.00,0.0,15.80
2,2,2015-04-21 15:21:04,2015-04-21 15:33:38,1,1.75,-73.951920,40.769421,1,N,-73.956421,40.787140,2,9.5,0.0,0.5,0.00,0.0,10.30
3,2,2015-01-29 18:58:54,2015-01-29 19:04:46,3,0.83,-74.009026,40.715710,1,N,-74.003418,40.723152,1,5.5,1.0,0.5,1.46,0.0,8.76
4,1,2015-06-05 10:47:29,2015-06-05 10:58:00,3,0.80,-73.985909,40.756176,1,N,-73.988358,40.747398,2,7.0,0.0,0.5,0.00,0.0,7.80


In [19]:
df_trips.info()
df_trips.columns
df_trips['VendorID'] = df_trips['VendorID'].astype('int8')
df_trips['passenger_count'] = df_trips['passenger_count'].astype('int8')
df_trips['RateCodeID'] = df_trips['RateCodeID'].astype('int8')
df_trips['payment_type'] = df_trips['payment_type'].astype('int8')
df_trips.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1500000 non-null  int64  
 1   tpep_pickup_datetime   1500000 non-null  object 
 2   tpep_dropoff_datetime  1500000 non-null  object 
 3   passenger_count        1500000 non-null  int64  
 4   trip_distance          1500000 non-null  float64
 5   pickup_longitude       1500000 non-null  float64
 6   pickup_latitude        1500000 non-null  float64
 7   RateCodeID             1500000 non-null  int64  
 8   store_and_fwd_flag     1500000 non-null  object 
 9   dropoff_longitude      1500000 non-null  float64
 10  dropoff_latitude       1500000 non-null  float64
 11  payment_type           1500000 non-null  int64  
 12  fare_amount            1500000 non-null  float64
 13  extra                  1500000 non-null  float64
 14  mta_tax           

In [20]:
df_trips[['fare_amount','dropoff_longitude', 'dropoff_latitude', 'pickup_longitude', 'pickup_latitude', 'extra', 'mta_tax', 'tip_amount','tolls_amount' ]] = df_trips[['fare_amount','dropoff_longitude', 'dropoff_latitude', 'pickup_longitude', 'pickup_latitude', 'extra', 'mta_tax', 'tip_amount','tolls_amount']].astype('float16')
df_trips['total_amount'] = df_trips['total_amount'].astype('float32')


In [21]:
df_trips['trip_distance'] = df_trips['trip_distance'].astype('float16')
df_trips.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1500000 non-null  int8   
 1   tpep_pickup_datetime   1500000 non-null  object 
 2   tpep_dropoff_datetime  1500000 non-null  object 
 3   passenger_count        1500000 non-null  int8   
 4   trip_distance          1500000 non-null  float16
 5   pickup_longitude       1500000 non-null  float16
 6   pickup_latitude        1500000 non-null  float16
 7   RateCodeID             1500000 non-null  int8   
 8   store_and_fwd_flag     1500000 non-null  object 
 9   dropoff_longitude      1500000 non-null  float16
 10  dropoff_latitude       1500000 non-null  float16
 11  payment_type           1500000 non-null  int8   
 12  fare_amount            1500000 non-null  float16
 13  extra                  1500000 non-null  float16
 14  mta_tax           

In [22]:
df_trips

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
0,1,2015-06-03 10:41:50,2015-06-03 11:08:33,1,2.800781,-73.9375,40.78125,1,N,-74.0000,40.75000,1,17.5,0.0,0.5,4.570312,0.0,22.870001
1,2,2015-02-14 19:38:55,2015-02-14 20:00:59,1,2.789062,-74.0000,40.71875,1,N,-74.0000,40.75000,2,15.0,0.0,0.5,0.000000,0.0,15.800000
2,2,2015-04-21 15:21:04,2015-04-21 15:33:38,1,1.750000,-73.9375,40.78125,1,N,-73.9375,40.78125,2,9.5,0.0,0.5,0.000000,0.0,10.300000
3,2,2015-01-29 18:58:54,2015-01-29 19:04:46,3,0.830078,-74.0000,40.71875,1,N,-74.0000,40.71875,1,5.5,1.0,0.5,1.459961,0.0,8.760000
4,1,2015-06-05 10:47:29,2015-06-05 10:58:00,3,0.799805,-74.0000,40.75000,1,N,-74.0000,40.75000,2,7.0,0.0,0.5,0.000000,0.0,7.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499995,1,2015-05-28 07:12:18,2015-05-28 07:15:19,1,0.700195,-74.0000,40.75000,1,N,-74.0000,40.75000,1,4.5,0.0,0.5,1.549805,0.0,6.850000
1499996,1,2015-01-18 21:19:45,2015-01-18 21:21:58,1,0.799805,-74.0000,40.75000,1,N,-74.0000,40.75000,1,4.5,0.5,0.5,1.000000,0.0,6.800000
1499997,1,2015-06-03 06:42:21,2015-06-03 06:52:14,1,2.000000,-73.9375,40.78125,1,N,-74.0000,40.75000,1,9.5,0.0,0.5,2.050781,0.0,12.350000
1499998,2,2015-03-09 19:58:57,2015-03-09 20:15:13,1,1.959961,-74.0000,40.71875,1,N,-74.0000,40.75000,1,12.0,1.0,0.5,2.759766,0.0,16.559999


In [23]:
df_trips_filtered = df_trips.loc[(df_trips['trip_distance'] < 21.0) & (df_trips['trip_distance'] > 0.2) & (df_trips['dropoff_longitude'] <= -73) & (df_trips['dropoff_longitude'] > -75) & (df_trips['dropoff_latitude'] >= 40) & (df_trips['dropoff_latitude'] < 42),:]


#

df_trips_filtered = df_trips_filtered.loc[(df_trips['pickup_longitude'] <= -73) & (df_trips['pickup_longitude'] > -75) & (df_trips['pickup_latitude'] >= 40) & (df_trips['pickup_latitude'] < 42),:]



In [24]:
def calculate_time_difference_in_hours(start_time: str, end_time: str) -> float:
    """
    Calculate the time difference in hours between two timestamps.

    :param start_time: The starting time as a string in the format 'YYYY-MM-DD HH:MM:SS'.
    :param end_time: The ending time as a string in the format 'YYYY-MM-DD HH:MM:SS'.
    :return: The time difference in hours as a float.
    """
    # Parse the input strings into datetime objects
    start = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
    end = datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S")

    # Calculate the time difference
    time_difference = end - start

    # Convert the time difference to hours
    hours_difference = time_difference.total_seconds() / 3600

    return hours_difference


In [25]:

df_trips_filtered["tpep_pickup_datetime"] = pd.to_datetime(df_trips_filtered["tpep_pickup_datetime"])
df_trips_filtered["tpep_dropoff_datetime"] = pd.to_datetime(df_trips_filtered["tpep_dropoff_datetime"])
df_trips_filtered['time_lentgh'] = (df_trips_filtered["tpep_dropoff_datetime"] - df_trips_filtered["tpep_pickup_datetime"]).dt.total_seconds() / 3600
df_trips_filtered = df_trips_filtered.drop(columns=['tpep_dropoff_datetime', 'tpep_pickup_datetime', 'store_and_fwd_flag'])
df_trips_filtered

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount,time_lentgh
0,1,1,2.800781,-73.9375,40.78125,1,-74.0000,40.75000,1,17.5,0.0,0.5,4.570312,0.0,22.870001,0.445278
1,2,1,2.789062,-74.0000,40.71875,1,-74.0000,40.75000,2,15.0,0.0,0.5,0.000000,0.0,15.800000,0.367778
2,2,1,1.750000,-73.9375,40.78125,1,-73.9375,40.78125,2,9.5,0.0,0.5,0.000000,0.0,10.300000,0.209444
3,2,3,0.830078,-74.0000,40.71875,1,-74.0000,40.71875,1,5.5,1.0,0.5,1.459961,0.0,8.760000,0.097778
4,1,3,0.799805,-74.0000,40.75000,1,-74.0000,40.75000,2,7.0,0.0,0.5,0.000000,0.0,7.800000,0.175278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499995,1,1,0.700195,-74.0000,40.75000,1,-74.0000,40.75000,1,4.5,0.0,0.5,1.549805,0.0,6.850000,0.050278
1499996,1,1,0.799805,-74.0000,40.75000,1,-74.0000,40.75000,1,4.5,0.5,0.5,1.000000,0.0,6.800000,0.036944
1499997,1,1,2.000000,-73.9375,40.78125,1,-74.0000,40.75000,1,9.5,0.0,0.5,2.050781,0.0,12.350000,0.164722
1499998,2,1,1.959961,-74.0000,40.71875,1,-74.0000,40.75000,1,12.0,1.0,0.5,2.759766,0.0,16.559999,0.271111


In [26]:
labels =df_trips_filtered['trip_distance']
x_train, x_test, y_train, y_test = train_test_split(df_trips_filtered, labels, test_size=0.2, random_state=42)

In [27]:
clf_rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)

clf_rf.fit(x_train, y_train)


RandomForestRegressor(n_jobs=-1, random_state=42)

In [29]:
predictions_rf = clf_rf.predict(x_test)

In [30]:
mae_rf = mean_absolute_error(predictions_rf, y_test)

print('random forest regression mae : ', mae_rf)

random forest regression mae :  1.6123539852216317e-08
